# Data Wrangling Template

In [1]:
import numpy as np
import pandas as pd
import requests
import os
import tweepy
from tweepy import OAuthHandler
import json
from timeit import default_timer as timer
import matplotlib.pyplot as plt

## Gather

WeRateDogs Twitter archive

- WeRateDogs Twitter archive contains basic tweet data for all 5000+ of their tweets, but not everything. One column the archive does contain though: each tweet's text, which I used to extract rating, dog name, and dog "stage" (i.e. doggo, floofer, pupper, and puppo) to make this Twitter archive "enhanced." Of the 5000+ tweets, I have filtered for tweets with ratings only (there are 2356).

In [107]:
#Import csv file
tae = pd.read_csv('twitter-archive-enhanced.csv')

In [3]:
#Create directory 
folder_name = 'image_predictions'
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

In [4]:
#Download file from url
url='https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'

response = requests.get(url)
response
with open(os.path.join(folder_name, url.split('/')[-1]), mode ='wb') as file:
    file.write(response.content)

In [5]:
#Import tsv file
ip = pd.read_csv('image_predictions/image-predictions.tsv', sep='\t')

Each tweet's retweet count and favorite ("like") count at minimum, and any additional data you find interesting. Using the tweet IDs in the WeRateDogs Twitter archive, query the Twitter API for each tweet's JSON data using Python's Tweepy library and store each tweet's entire set of JSON data in a file called tweet_json.txt file. Each tweet's JSON data should be written to its own line. Then read this .txt file line by line into a pandas DataFrame with (at minimum) tweet ID, retweet count, and favorite count. 


Image Predictions File

- The results: a table full of image predictions (the top three only) alongside each tweet ID, image URL, and the image number that corresponded to the most confident prediction (numbered 1 to 4 since tweets can have up to four images).

- tweet_id is the last part of the tweet URL after "status/" → https://twitter.com/dog_rates/status/889531135344209921
- p1 is the algorithm's #1 prediction for the image in the tweet → golden retriever
- p1_conf is how confident the algorithm is in its #1 prediction → 95%
- p1_dog is whether or not the #1 prediction is a breed of dog → TRUE
- p2 is the algorithm's second most likely prediction → Labrador retriever
- p2_conf is how confident the algorithm is in its #2 prediction → 1%
- p2_dog is whether or not the #2 prediction is a breed of dog → TRUE



## Assess

### Assessment of Twitter Archive Enhanced

In [6]:
tae.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None


In [7]:
tae.sample(5)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
1879,675015141583413248,NaN,NaN,2015-12-10 18:12:05 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Meet Moe. He's a golden Fetty Woof. Doesn't re...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/675015141...,10,10,Moe,None,None,None,None
1021,746726898085036033,NaN,NaN,2016-06-25 15:29:00 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Meet Abby. She's incredibly distracting. Just ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/746726898...,12,10,Abby,None,None,None,None
1820,676588346097852417,NaN,NaN,2015-12-15 02:23:26 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Bubbles. He kinda resembles a fish. Al...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/676588346...,5,10,Bubbles,None,None,None,None
29,886366144734445568,NaN,NaN,2017-07-15 23:25:31 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Roscoe. Another pupper fallen victim t...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/886366144...,12,10,Roscoe,None,None,pupper,None
1471,693993230313091072,NaN,NaN,2016-02-01 03:04:14 +0000,"<a href=""http://vine.co"" rel=""nofollow"">Vine -...",These lil fellas are the best of friends. 12/1...,NaN,NaN,NaN,https://vine.co/v/i5ETazP5hrm,12,10,None,None,None,None,None


In [8]:
tae.describe()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,retweeted_status_id,retweeted_status_user_id,rating_numerator,rating_denominator
count,2.356000e+03,7.800000e+01,7.800000e+01,1.810000e+02,1.810000e+02,2356.000000,2356.000000
mean,7.427716e+17,7.455079e+17,2.014171e+16,7.720400e+17,1.241698e+16,13.126486,10.455433
std,6.856705e+16,7.582492e+16,1.252797e+17,6.236928e+16,9.599254e+16,45.876648,6.745237
min,6.660209e+17,6.658147e+17,1.185634e+07,6.661041e+17,7.832140e+05,0.000000,0.000000
25%,6.783989e+17,6.757419e+17,3.086374e+08,7.186315e+17,4.196984e+09,10.000000,10.000000
50%,7.196279e+17,7.038708e+17,4.196984e+09,7.804657e+17,4.196984e+09,11.000000,10.000000
75%,7.993373e+17,8.257804e+17,4.196984e+09,8.203146e+17,4.196984e+09,12.000000,10.000000
max,8.924206e+17,8.862664e+17,8.405479e+17,8.874740e+17,7.874618e+17,1776.000000,170.000000


In [9]:
tae.info()

# tweet_id string?
# timestamp time
# source change text
# text analysis
# retweeted timestamp time
# name analysis
# doggo, floofer, pupper, puppo --> one column

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   tweet_id                    2356 non-null   int64  
 1   in_reply_to_status_id       78 non-null     float64
 2   in_reply_to_user_id         78 non-null     float64
 3   timestamp                   2356 non-null   object 
 4   source                      2356 non-null   object 
 5   text                        2356 non-null   object 
 6   retweeted_status_id         181 non-null    float64
 7   retweeted_status_user_id    181 non-null    float64
 8   retweeted_status_timestamp  181 non-null    object 
 9   expanded_urls               2297 non-null   object 
 10  rating_numerator            2356 non-null   int64  
 11  rating_denominator          2356 non-null   int64  
 12  name                        2356 non-null   object 
 13  doggo                       2356 

In [10]:
# access rows
tae.iloc[1224:1225]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
1224,714214115368108032,NaN,NaN,2016-03-27 22:14:49 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Happy Easter from the squad! 🐇🐶 13/10 for all ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/714214115...,13,10,None,None,None,None,None


In [11]:
# access columns
tae[['tweet_id', 'timestamp']]

,tweet_id,timestamp
0,892420643555336193,2017-08-01 16:23:56 +0000
1,892177421306343426,2017-08-01 00:17:27 +0000
2,891815181378084864,2017-07-31 00:18:03 +0000
3,891689557279858688,2017-07-30 15:58:51 +0000
4,891327558926688256,2017-07-29 16:00:24 +0000
...,...,...
2351,666049248165822465,2015-11-16 00:24:50 +0000
2352,666044226329800704,2015-11-16 00:04:52 +0000
2353,666033412701032449,2015-11-15 23:21:54 +0000
2354,666029285002620928,2015-11-15 23:05:30 +0000


In [12]:
# build subset of table
tae[['tweet_id', 'timestamp']].iloc[1224:1228]

,tweet_id,timestamp
1224,714214115368108032,2016-03-27 22:14:49 +0000
1225,714141408463036416,2016-03-27 17:25:54 +0000
1226,713919462244790272,2016-03-27 02:43:58 +0000
1227,713909862279876608,2016-03-27 02:05:49 +0000


In [13]:
# build new dataframe
data = tae[['tweet_id', 'timestamp']].iloc[1224:1228]

In [14]:
# query dataframe
data.query('tweet_id == 714214115368108032')

,tweet_id,timestamp
1224,714214115368108032,2016-03-27 22:14:49 +0000


In [17]:
columns = tae.columns
columns

Index(['tweet_id', 'in_reply_to_status_id', 'in_reply_to_user_id', 'timestamp',
       'source', 'text', 'retweeted_status_id', 'retweeted_status_user_id',
       'retweeted_status_timestamp', 'expanded_urls', 'rating_numerator',
       'rating_denominator', 'name', 'doggo', 'floofer', 'pupper', 'puppo'],
      dtype='object')

In [97]:
(len(tae)) - (tae.in_reply_to_status_id.isna().sum())

78

In [26]:
long_columns = []

for column in columns:
    length = len(column)
    if length > 10:
        long_columns.append(column)    

In [32]:
long_columns

['in_reply_to_status_id',
 'in_reply_to_user_id',
 'retweeted_status_id',
 'retweeted_status_user_id',
 'retweeted_status_timestamp',
 'expanded_urls',
 'rating_numerator',
 'rating_denominator']

In [33]:
id_columns = []

for column in columns:
    if 'id' in column:
        id_columns.append(column)    

In [34]:
id_columns

['tweet_id',
 'in_reply_to_status_id',
 'in_reply_to_user_id',
 'retweeted_status_id',
 'retweeted_status_user_id']

In [108]:
meet_list = []

for i in range(0, len(tae)):
    if 'meet' in tae.text.iloc[i]:
        meet = tae[['tweet_id', 'text']].iloc[i]
        meet_list.append(meet)

In [109]:
meet_list

[tweet_id                                   886680336477933568
 text        This is Derek. He's late for a dog meeting. 13...
 Name: 28, dtype: object,
 tweet_id                                   884162670584377345
 text        Meet Yogi. He doesn't have any important dog m...
 Name: 43, dtype: object,
 tweet_id                                   779123168116150273
 text        This is Reggie. He hugs everyone he meets. 12/...
 Name: 750, dtype: object,
 tweet_id                                   669353438988365824
 text        This is Tessa. She is also very pleased after ...
 Name: 2169, dtype: object,
 tweet_id                                   667806454573760512
 text        This is Filup. He is overcome with joy after f...
 Name: 2251, dtype: object]

In [110]:
meet_df = pd.DataFrame(meet_list, columns = ['tweet_id', 'text'])
meet_df

,tweet_id,text
28,886680336477933568,This is Derek. He's late for a dog meeting. 13...
43,884162670584377345,Meet Yogi. He doesn't have any important dog m...
750,779123168116150273,This is Reggie. He hugs everyone he meets. 12/...
2169,669353438988365824,This is Tessa. She is also very pleased after ...
2251,667806454573760512,This is Filup. He is overcome with joy after f...


In [114]:
for i in range(0, len(tae)):
    if 'meat' in tae.text.iloc[i]:
        print(tae.text.iloc[i])

"So... we meat again" (I'm so sorry for that pun I couldn't resist pls don't unfollow) 10/10 https://t.co/XFBrrqapZa


In [143]:
# new column needs to be created
tae.text_meat = tae.text.str.replace('meet', 'meat')

<ipython-input-143-ca4b81f5d35a>:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  tae.text_meat = tae.text.str.replace('meet', 'meat')


In [144]:
meat_list = []

for i in range(0, len(tae)):
    if 'meat' in tae.text_meat.iloc[i]:
        meat = tae[['tweet_id', 'text']].iloc[i]
        meat_list.append(meat)

In [145]:
meat_df = pd.DataFrame(meat_list, columns = ['tweet_id', 'text'])
meat_df

,tweet_id,text
28,886680336477933568,This is Derek. He's late for a dog meeting. 13...
43,884162670584377345,Meet Yogi. He doesn't have any important dog m...
750,779123168116150273,This is Reggie. He hugs everyone he meets. 12/...
995,748346686624440324,"""So... we meat again"" (I'm so sorry for that p..."
2169,669353438988365824,This is Tessa. She is also very pleased after ...
2251,667806454573760512,This is Filup. He is overcome with joy after f...


### Assessment of Image Predictions

### Issues Twitter Archive Enhanced

#### Tidiness

#### Quality


### Issues Image Predictions

#### Tidiness

#### Quality


# Clean


## Clean Twitter Archive Enhanced

#### Define

#### Code

#### Test

# Store, Analyze, and Visualize